#### SQLite port_lite database: sales, stocks table
#### csv files: daily-watch-list, daily-sales, monitors, stocks-all

In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 6, 12)

In [2]:
#today = date(2023, 5, 3)

### Yesterday = Last business day

In [3]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-06-12
yesterday: 2023-06-09


### Restart and Run All Cells | daily-watch-list & daily-sales upload to obsidian

In [4]:
#cols = 'fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','chg_acc':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','%chg_acc':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [5]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1265677,TTB,2023-06-09,2023-06-09,0,1.66,1.66,0,0.00%,0,0,1.69,1.65,"403,425,915",2023-06-11,2023-06-11,1


In [6]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [7]:
name = names.to_string(index=False)
type(name)

str

In [8]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'TTB'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9717,TTB,0.00,0.00,O,1.09,0.00,0.00,0.00,0.00,"20,000",-9,17,0,0,0,52WL,SET50


### End of Tables in the process

In [9]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-06-09'
ORDER BY a.name



,name
0,AH
1,AWC
2,BANPU
3,BBL
4,BCH
5,BDMS
6,BEM
7,BH
8,CPNREIT
9,EA


In [10]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-06-09' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [11]:
df = pd.read_sql(sql, conlite)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('chg_acc = to_price - fm_price',inplace=True)
df['%chg_acc'] = round(df['chg_acc']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,chg_acc,%chg_acc
0,BANPU,2023-06-02,2023-06-09,8.30,8.75,"502,601,798",8.80,8.20,B,SET50,0.45,5.42%
1,IVL,2023-06-08,2023-06-09,33.25,33.25,"34,539,645",33.75,32.00,B,SET50,0.00,0.00%
2,MCS,2023-06-08,2023-06-09,7.00,7.00,"2,497,461",7.15,6.95,B,SET,0.00,0.00%
3,SINGER,2023-06-08,2023-06-09,11.20,11.20,"22,419,726",11.60,10.70,B,SET100,0.00,0.00%
4,STA,2023-06-09,2023-06-09,18.80,18.80,"5,472,715",19.00,18.60,B,SET100,0.00,0.00%
5,SYNEX,2023-06-08,2023-06-09,13.00,13.30,"462,153",13.30,12.80,B,SET,0.30,2.31%
6,TMT,2023-06-09,2023-06-09,7.50,7.50,"239,238",7.50,7.45,B,SET,0.00,0.00%
7,AH,2023-06-08,2023-06-09,36.50,34.50,"103,830,264",37.75,27.75,I,SET,-2.00,-5.48%
8,BCH,2023-06-09,2023-06-09,18.30,18.30,"14,152,226",18.60,18.30,I,SET100,0.00,0.00%
9,CPNREIT,2023-06-09,2023-06-09,13.00,13.00,"2,498,945",13.00,12.70,I,SET,0.00,0.00%


### Price change up or down more than 3%

In [12]:
# For each row in the DataFrame, generate a series of dates between the start and end dates
df['dates'] = df.apply(lambda x: pd.date_range(x['fm_date'], x['to_date']), axis=1)

# For each row in the DataFrame, select only the business days (Monday through Friday) from the series of dates
df['business_days'] = df['dates'].apply(lambda x: x[x.isin(pd.date_range(x.min(), x.max(), freq='B'))])

# For each row in the DataFrame, count the number of business days between the start and end dates
df['bdays'] = df['business_days'].apply(lambda x: len(x))

In [13]:
colt =  'name %chg_acc status fm_price to_price market \
fm_date bdays max_price min_price'.split()
watch_list = df.loc[abs(df['%chg_acc']) >= 3,colt]
watch_list.sort_values('%chg_acc',ascending=False)
watch_list

,name,%chg_acc,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
0,BANPU,5.42,B,8.30,8.75,SET50,2023-06-02,6,8.80,8.20
7,AH,-5.48,I,36.50,34.50,SET,2023-06-08,2,37.75,27.75
20,JMT,8.86,O,39.50,43.00,SET50,2023-06-07,3,44.00,38.50
23,KTC,3.92,O,51.00,53.00,SET50,2023-06-07,3,53.00,50.00
31,NER,4.02,S,4.48,4.66,SET,2023-06-07,3,4.68,4.44
32,PTTGC,5.48,S,36.50,38.50,SET50,2023-06-07,3,38.50,35.50


In [14]:
# cannot use with column that starts with %
#df.query('abs(%chg_acc) >= 1.0')

In [15]:
file_name = "daily-watch-list.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(output_file, index=False)
watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(data_file, index=False)
watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(data_file, index=False)
watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(one_file, index=False)
watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(osd_file, index=False)

### Create daily-sales from sales

In [16]:
df[colt].style.format(format_dict)

,name,%chg_acc,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
0,BANPU,5.42%,B,8.30,8.75,SET50,2023-06-02,6,8.80,8.20
1,IVL,0.00%,B,33.25,33.25,SET50,2023-06-08,2,33.75,32.00
2,MCS,0.00%,B,7.00,7.00,SET,2023-06-08,2,7.15,6.95
3,SINGER,0.00%,B,11.20,11.20,SET100,2023-06-08,2,11.60,10.70
4,STA,0.00%,B,18.80,18.80,SET100,2023-06-09,1,19.00,18.60
5,SYNEX,2.31%,B,13.00,13.30,SET,2023-06-08,2,13.30,12.80
6,TMT,0.00%,B,7.50,7.50,SET,2023-06-09,1,7.50,7.45
7,AH,-5.48%,I,36.50,34.50,SET,2023-06-08,2,37.75,27.75
8,BCH,0.00%,I,18.30,18.30,SET100,2023-06-09,1,18.60,18.30
9,CPNREIT,0.00%,I,13.00,13.00,SET,2023-06-09,1,13.00,12.70


In [17]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[colt].sort_values(['%chg_acc'],ascending=[True]).to_csv(output_file, index=False)
df[colt].sort_values(['%chg_acc'],ascending=[True]).to_csv(data_file, index=False)
df[colt].sort_values(['%chg_acc'],ascending=[True]).to_csv(one_file, index=False)
df[colt].sort_values(['%chg_acc'],ascending=[True]).to_csv(osd_file, index=False)

### Create monitors from orders

In [18]:
sql = """
SELECT name, trade 
FROM orders 
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors.sample(5)

,trade
name,
CPNREIT,S
BCH,S
TFFIF,S
SYNEX,B
AH,S


In [19]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,24
1,S,10


### output_file = iCloud Drive on PC

In [20]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(output_file)
monitors.to_csv(one_file)

### End of daily process

In [21]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,24
1,S,10


In [22]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
BANPU,B,SET50
IVL,B,SET50
MCS,B,SET
SINGER,B,SET100
STA,B,SET100
...,...,...
TCAP,X,SET100
TFG,X,SET
TOP,X,SET50


In [23]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(one_file)

In [24]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("B")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,trade
name,
AWC,B
BANPU,B
BBL,B
BDMS,B
BEM,B
BH,B
EA,B
ICHI,B
IVL,B


In [25]:
buy_candidates.shape[0]

24

In [26]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("S")
ORDER BY name
"""
sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,trade
name,
AH,S
BCH,S
CPNREIT,S
GVREIT,S
JASIF,S
JMART,S
NER,S
PTTGC,S
SENA,S


In [27]:
sell_candidates.shape[0]

10